In [157]:
# Add PyDatAnalysis to path
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, "/Users/owensheekey/Documents/Research/PyDatAnalysis")

export_path = 'Exports/'

In [223]:
from __future__ import annotations
from progressbar import progressbar
from src.DatObject.Make_Dat import get_dat, get_dats
import src.UsefulFunctions as U
from src.DataStandardize.ExpSpecific.Feb21 import Feb21Exp2HDF, Feb21ExpConfig
from src.DataStandardize.ExpConfig import ExpConfigGroupDatAttribute, ExpConfigBase
import multiprocessing as mp
import plotly.graph_objs as go
import numpy as np
import lmfit as lm
from typing import TYPE_CHECKING, Iterable, Optional
from src.DatObject.Attributes.Transition import i_sense_digamma, i_sense, i_sense_digamma_quad
from src.UsefulFunctions import edit_params
from src.DatObject.Attributes.SquareEntropy import square_wave_time_array, integrate_entropy
import logging
logger = logging.getLogger(__name__)
import src.UsefulFunctions as U

In [226]:
# dats = get_dats(range(1637, 1653), overwrite=True)
#dats = get_dats(range(1653, 1669), overwrite=True)
#dats = get_dats(range(1669, 1685), overwrite=True)

#dats = get_dats(range(1778, 1795))
dats = get_dats(range(1798, 1803))

In [207]:
def narrow_fit(dat, width, **kwargs):
    '''
    Get a fit only including +/- width in dat.x around center of transition
    kwargs is the stuff to pass to get_fit
    Return a fit
    '''
    x = np.copy(dat.SquareEntropy.avg_x)
    y = np.copy(dat.SquareEntropy.default_Output.averaged)
    y = np.mean(y[(0, 2), :], axis=0)

    start_ind = np.nanargmin(np.abs(np.add(x, width)))
    end_ind = np.nanargmin(np.abs(np.subtract(x, width)))

    x[:start_ind] = [np.nan] * start_ind
    x[end_ind:] = [np.nan] * (len(x) - end_ind)

    y[:start_ind] = [np.nan] * start_ind
    y[end_ind:] = [np.nan] * (len(y) - end_ind)

    fit = dat.SquareEntropy.get_fit(
        x=x,
        data=y,
        **kwargs)
    return fit

In [224]:
def do_calc(datnum, overwrite=True):
    """Just a function which can be passed to a process pool for faster calculation"""
    save_name = 'SPS.005'

    dat = get_dat(datnum)

    setpoints = [0.005, None]

    # Get other inputs
    setpoint_times = square_wave_time_array(dat.SquareEntropy.square_awg)
    sp_start, sp_fin = [U.get_data_index(setpoint_times, sp) for sp in setpoints]
    logger.debug(f'Setpoint times: {setpoints}, Setpoint indexs: {sp_start, sp_fin}')

    # Run Fits
    pp = dat.SquareEntropy.get_ProcessParams(name=None,  # Load default and modify from there
                                             setpoint_start=sp_start, setpoint_fin=sp_fin,
                                             transition_fit_func=i_sense,
                                             save_name=save_name)
    out = dat.SquareEntropy.get_Outputs(name=save_name, inputs=None, process_params=pp, overwrite=overwrite)
    dat.Entropy.get_fit(which='avg', name=save_name, data=out.average_entropy_signal, x=out.x, check_exists=False,
                        overwrite=overwrite)
    [dat.Entropy.get_fit(which='row', row=i, name=save_name,
                         data=row, x=out.x, check_exists=False,
                         overwrite=overwrite) for i, row in enumerate(out.entropy_signal)]
    return True

In [229]:
Θ = dats[0].SquareEntropy.avg_fit.best_values.theta 
fit = dats[0].SquareEntropy.get_fit(which='avg', which_fit='transition', transition_part='cold', check_exists=False)
params = fit.params
params.add('g', value=0, vary=True, min=-50, max=1000)
new_pars = edit_params(params, param_name='theta', value=Θ, vary=False) # Based on some old DC bias scans

In [230]:
narrow_fit(
    dats[0],
    400,
    initial_params=new_pars,
    fit_func=i_sense_digamma,
    check_exists=False).best_values.mid

-0.002760976692256251

In [231]:
amp_digamma_ = [narrow_fit(
    dat,
    400,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.amp
for dat in progressbar(dats)]

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [232]:
g_digamma_ = [narrow_fit(
    dat,
    400,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.g
for dat in progressbar(dats)]

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [233]:
theta_digamma_ = [narrow_fit(
    dat,
    400,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.theta
for dat in progressbar(dats)]

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [234]:
mids_digamma_ = [narrow_fit(
    dat,
    400,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.mid
for dat in progressbar(dats)]

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [235]:
amp_digamma_ = [narrow_fit(
    dat,
    400,
    initial_params=new_pars, 
    fit_func=i_sense_digamma, 
    check_exists=False).best_values.amp
for dat in progressbar(dats)]

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [236]:
fig = go.Figure()
datnums = [dat.datnum for dat in dats]
escs = [dat.Logs.fds["ESC"] for dat in dats]

fig.add_trace(go.Scatter(mode='markers', x=escs, y=amp_digamma_, text=[dat.datnum for dat in dats], name="Data"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Amplitude /nA',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
res = np.polyfit(escs, amp_digamma_, 2)
fit = np.polyval(res, escs)
print(res)
fig.add_trace(go.Scatter(mode='lines', x=escs, y=fit, name="Fit"))
fig.show(renderer="browser")

[-6.08056885e-05 -3.93451364e-02 -5.40995556e+00]


In [145]:
#fig.write_html("Exports/Set1_Ampl_03_07.html")

In [237]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=g_digamma_/Θ, text=[dat.datnum for dat in dats], name="Data"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Gamma/Theta /arb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')

fig.show(renderer="browser")

In [147]:
#fig.write_html("Exports/Set1_Gamma_03_07.html")

In [238]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode='markers', x=escs, y=theta_digamma_, text=[dat.datnum for dat in dats], name="Data"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Theta /mV',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')

fig.show(renderer="browser")

In [239]:
fig = go.Figure()
for dat in progressbar(dats):
    x = dat.SquareEntropy.avg_x[::10]
    y = dat.SquareEntropy.default_Output.averaged
    y = np.mean(y[(0, 2), :], axis=0)[::10]
    xfit = np.linspace(-500,500,1001)
    fit = narrow_fit(
            dat,
            400,
            initial_params=new_pars, 
            fit_func=i_sense_digamma, 
            check_exists=False)
    yfit = fit.eval_fit(xfit)
    fig.add_trace(go.Scatter(mode='markers', x=x, y=y, name=f'{dat.datnum}d'))
    fig.add_trace(go.Scatter(mode='lines', x=xfit, y=yfit, name=f'{dat.datnum}f', marker_color='grey'))
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Current /nA',
                      title=f'Dat {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer="browser")

100% (5 of 5) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [149]:
#fig.write_html("Exports/Set1_03_07_narrow_fitting.html")

In [49]:
def get_deltaT(dat):
    """Returns deltaT of a given dat in mV"""
    ho1 = dat.AWG.max(0)  # 'HO1/10M' gives nA * 10
    t = dat.Logs.temps.mc

    # Datnums to search through (only thing that should be changed)
    datnums = set(range(1312, 1451+1)) - set(range(1312, 1451+1, 4))
    # datnums = set()
    # for j in range(5):
    #     datnums = datnums.union(set(range(28 * j + 1312, 28 * j + 1312 + 4 * 7 + 1)) - set([28 * j + 1312 + 4 * i for i in range(8)]))
    # datnums = list(datnums)

    dats = get_dats(datnums)

    dats = [d for d in dats if np.isclose(d.Logs.temps.mc, dat.Logs.temps.mc, rtol=0.1)]  # Get all dats where MC temp is within 10%
    bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])

    indp = np.argmin(abs(bias_lookup - ho1))
    indm = np.argmin(abs(bias_lookup + ho1))
    theta_z = np.nanmean([d.Transition.avg_fit.best_values.theta for d in dats if d.Logs.fds['HO1/10M'] == 0])

    # temp_lookup = np.array([d.Logs.temps.mc for d in dats])
    # bias_lookup = np.array([d.Logs.fds['HO1/10M'] for d in dats])
    #
    # indp = np.argmin(temp_lookup - t + bias_lookup - ho1)
    # indm = np.argmin(temp_lookup - t + bias_lookup + ho1)
    # indz = np.argmin(temp_lookup - t + bias_lookup)

    theta_p = dats[indp].Transition.avg_fit.best_values.theta
    theta_m = dats[indm].Transition.avg_fit.best_values.theta
    # theta_z = dats[indz].Transition.avg_fit.best_values.theta
    return (theta_p + theta_m) / 2 - theta_z

In [240]:
deltaT = [get_deltaT(dat) for dat in dats]
ampl = amp_digamma_
for i, dat in enumerate(dats):
    dat.Entropy.set_integration_info(dT=deltaT[i], amp=ampl[i], overwrite=True)

In [241]:
fig = go.Figure()
int_ents = []
for i in range(len(dats)):
    int_ent = integrate_entropy(dats[i].SquareEntropy.avg_entropy_signal, dats[i].Entropy.integration_info.sf)
    int_ents.append(int_ent[-1])
    fig.add_trace(go.Scatter(mode='markers', 
                             x=np.subtract(dats[i].SquareEntropy.avg_x, mids_digamma_[i]), 
                             y=int_ent,
                             name= f'Gamma/Theta /arb = {np.divide(g_digamma_,Θ)[i]:.2f}'))

fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(2), np.log(2)], name="Log2"))  
fig.add_trace(go.Scatter(mode='lines', x=[-400,400], y=[np.log(3), np.log(3)], name="Log3"))  
fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')

In [152]:
#fig.write_html("Exports/Set1_03_07_narrow_fitting_allentropy.html")

In [242]:
for dn in datnums:
    do_calc(dn)
ents = [dat.Entropy.get_fit(which='avg', name="SPS.005", check_exists=False).best_values.dS for dat in dats]
for i in range(len(ents)):
    print(f'{datnums[i]}:{ents[i]:.4f}')

1798:0.7397
1799:0.7986
1800:0.7860
1801:0.8762
1802:0.7346


In [243]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='markers+lines', x=escs, y=int_ents, text=[dat.datnum for dat in dats], name="Int"))
fig.add_trace(go.Scatter(mode='markers+lines', x=escs, y=ents, text=[dat.datnum for dat in dats], name="Fit"))
fig.update_layout(xaxis_title='ESC/ mV', yaxis_title='Fit Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')

fig.show(renderer="browser")

In [244]:
fig = go.Figure()
int_ents = []

for i, dat in enumerate(dats):
    ent = dat.Entropy.get_fit(which='avg', name="SPS.005", check_exists=True).best_values.dS
    xfit = np.linspace(-500,500,1001)
    entyfit = dat.Entropy.get_fit(which='avg', name="SPS.005", check_exists=True).eval_fit(xfit)

    fig.add_trace(go.Scatter(mode='markers', 
                             x=dat.SquareEntropy.avg_x, 
                             y=dat.SquareEntropy.avg_entropy_signal,
                             name= f'Gamma/Theta /arb = {np.divide(g_digamma_,Θ)[i]:.2f}'))
    fig.add_trace(go.Scatter(mode='lines', 
                             x=xfit, 
                             y=entyfit, 
                             name= f'delta S = {ent:.2f}',
                             marker_color='grey'))

fig.update_layout(xaxis_title='ACC/100 /mV', yaxis_title='Entropy /kb',
                      title=f'Dats {dats[0].datnum} - {dats[-1].datnum}')
fig.show(renderer='browser')

Fit found with same initial arguments, but hash does not match. Recalculating fit now
